In [ ]:
import os
import json
from model import modelRecruitment

with open('../config.json','rb') as file:
    config = json.load(file)

path_to_model = os.getcwd() + "/model"

In [ ]:
from pymongo import MongoClient

host = config['mongodb']['host']
port = config['mongodb']['port']
user = config['mongodb']['user']
password = config['mongodb']['password']

try:
    server = MongoClient(f'mongodb://{user}:{password}@{host}:{port}/')
    db = server.admin
    server_status = db.command("ping")
    print("MongoDB connection successful:", server_status)

    databases = server.list_database_names()
    print("Databases:", databases)

except Exception as e:
    print("An error occurred:", e)

In [ ]:
import pandas as pd

def predict_row(row):
    data_predict = row.to_dict()
    return modelRecruitment.runModel(data_predict, path_to_model)

db = server["ftde3"]
collection = db["kelompok2_data_recruitment_selection_raw"]

data = pd.DataFrame(list(collection.find()))

columns_to_drop = ['_id', 'CandidateID', 'Name', 'ApplicationDate', 'InterviewDate']
data_predict = data.drop(columns=columns_to_drop, axis=1)

data['Predict'] = data_predict.apply(predict_row, axis=1)

In [ ]:
new_data = data.drop(columns=['_id']).to_dict(orient='records')

db = server["ftde3"]
collection = db["kelompok2_data_recruitment_selection"]

collection.insert_many(new_data)